In [2]:
import pandas as pd 
data = pd.read_csv("traffic_stops.csv", low_memory=False)

In [4]:
#Stop Date format changed from YYYY-MM-DD to DD-MM-YYYY
df = pd.read_csv('traffic_stops.csv',low_memory=False)
df['stop_date'] = pd.to_datetime(df['stop_date'], format='%Y-%m-%d').dt.strftime('%d-%m-%Y')
df.to_csv('traffic_stops_new.csv', index=False)
print("Date format converted")
print("Saved as traffic_stops_new.csv")

Date format converted
Saved as traffic_stops_new.csv


In [5]:
#Remove the columns that only contains missing value
df = df.replace('', pd.NA)
columns_to_drop = df.columns[df.isna().all()]
df_cleaned = df.drop(columns=columns_to_drop)
df_cleaned.to_csv('traffic_stops_cleaned.csv', index=False)
print(f"Original shape: {df.shape}")
print(f"Columns removed: {list(columns_to_drop)}")
print(f"New shape: {df_cleaned.shape}")
print("Cleaned file saved as traffic_stops_cleaned.csv")

Original shape: (65538, 16)
Columns removed: []
New shape: (65538, 16)
Cleaned file saved as traffic_stops_cleaned.csv


In [6]:
#NaN inspection
df = pd.read_csv("traffic_stops_cleaned.csv",low_memory=False).replace('', pd.NA)
print("Missing values per column:")
print(df.isna().sum())
print("\nPercentage missing:")
print(df.isna().mean().round(4) * 100)

Missing values per column:
stop_date                 0
stop_time                 0
country_name              0
driver_gender             0
driver_age_raw            0
driver_age                0
driver_race               0
violation_raw             0
violation                 0
search_conducted          0
search_type           21720
stop_outcome              0
is_arrested               0
stop_duration             0
drugs_related_stop        0
vehicle_number            0
dtype: int64

Percentage missing:
stop_date              0.00
stop_time              0.00
country_name           0.00
driver_gender          0.00
driver_age_raw         0.00
driver_age             0.00
driver_race            0.00
violation_raw          0.00
violation              0.00
search_conducted       0.00
search_type           33.14
stop_outcome           0.00
is_arrested            0.00
stop_duration          0.00
drugs_related_stop     0.00
vehicle_number         0.00
dtype: float64


In [7]:
#Search_type NaN Handling
df = pd.read_csv("traffic_stops_cleaned.csv",low_memory=False)
df['was_searched'] = df['search_conducted'].copy()
df['search_type'] = df['search_type'].fillna('No Search')  # For all records
print("Missing values after NaN Handling:")
print(df.isna().sum())
df.to_csv('c', index=False)

Missing values after NaN Handling:
stop_date             0
stop_time             0
country_name          0
driver_gender         0
driver_age_raw        0
driver_age            0
driver_race           0
violation_raw         0
violation             0
search_conducted      0
search_type           0
stop_outcome          0
is_arrested           0
stop_duration         0
drugs_related_stop    0
vehicle_number        0
was_searched          0
dtype: int64


In [8]:
df

,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number,was_searched
0,01-01-2020,0:00:00,Canada,M,59,19,Asian,Drunk Driving,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473,True
1,01-01-2020,0:01:00,India,M,35,58,Other,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441,False
2,01-01-2020,0:02:00,USA,M,26,76,Black,Signal Violation,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264,False
3,01-01-2020,0:03:00,Canada,M,26,76,Black,Speeding,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807,True
4,01-01-2020,0:04:00,Canada,M,62,75,Other,Speeding,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65533,15-02-2020,12:13:00,India,F,54,48,Black,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,DL56GW6568,False
65534,15-02-2020,12:14:00,Canada,F,18,35,Hispanic,Seatbelt,Other,True,Vehicle Search,Ticket,False,16-30 Min,True,TN73EO7098,True
65535,15-02-2020,12:15:00,USA,M,27,41,Asian,Seatbelt,DUI,True,Frisk,Ticket,True,30+ Min,True,GJ33MX8328,True
65536,15-02-2020,12:16:00,Canada,F,49,63,Black,Seatbelt,Other,False,No Search,Warning,True,0-15 Min,True,KA24UZ8488,False


In [23]:
import psycopg2

# Render connection details
conn_postgres = psycopg2.connect(
    host="dpg-d178l9gdl3ps73a5q1mg-a.singapore-postgres.render.com",
    database="trafficdb_q63e",
    user="thariga_ds",
    password="DUyvvsd9XN1ZYXsYcxM5J6Y9L6Yzt8yl",
    port="5432"
)

cursor_postgres = conn_postgres.cursor()
print("PostgreSQL connection to Render established!")


PostgreSQL connection to Render established!


In [24]:
import pandas as pd
from sqlalchemy import create_engine, text

# ✅ Render PostgreSQL credentials
host = "dpg-d178l9gdl3ps73a5q1mg-a.singapore-postgres.render.com"
port = "5432"
database = "trafficdb_q63e"
username = "thariga_ds"
password = "DUyvvsd9XN1ZYXsYcxM5J6Y9L6Yzt8yl"

# ✅ SQLAlchemy connection string for Render cloud PostgreSQL
engine_string = f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}"

# ✅ Create engine
engine = create_engine(engine_string)

# ✅ Test connection
def create_connection():
    try:
        with engine.connect() as conn:
            result = conn.execute(text("SELECT 1"))
            print("✅ Connection successful:", result.fetchone())
    except Exception as e:
        print("Connection failed:", e)

# ✅ Upload DataFrame to PostgreSQL table
try:
    df.to_sql("traffic_stops", engine, if_exists='replace', index=False)
    print("✅ Data pushed to PostgreSQL table: traffic_stops")
except Exception as e:
    print("Failed to push data:", e)


✅ Data pushed to PostgreSQL table: traffic_stops
